# Phonopy workflow ideas

In [1]:
%%time
%config IPCompleter.evaluation='unsafe'

import matplotlib.pylab as plt
from pyiron_workflow.workflow import Workflow
from pyiron_workflow.function import single_value_node

CPU times: user 217 ms, sys: 42.7 ms, total: 259 ms
Wall time: 371 ms


In [2]:
%%time
wf = Workflow('phonopy')
wf.register('structure_transform', 'pyiron_workflow.node_library.structure_transform')
wf.register('structure_build', 'pyiron_workflow.node_library.structure_build')
wf.register('phonopy', 'pyiron_workflow.node_library.phonopy')

CPU times: user 3.42 s, sys: 447 ms, total: 3.87 s
Wall time: 4.69 s


In [3]:
from pyiron_workflow.node_library.phonopy import InputPhonopyGenerateSupercells

In [4]:
@Workflow.wrap_as.macro_node(
    "imaginary_modes", "total_dos", "energy_relaxed", "energy_initial", "energy_displaced"
)
def run_phonopy(wf, element: str, cell_size: int = 2, vacancy_index: int | None = None, displacement: float = 0.01):
    wf.structure = wf.create.structure_build.cubic_bulk_cell(
        element=element, cell_size=cell_size, vacancy_index=vacancy_index
    )
    # explicit output needed since macro and not single_value_node (we should have also a single_value_macro)
    wf.relaxed_structure = wf.create.phonopy.calc_minimize(
        atoms=wf.structure.outputs.structure
    )

    wf.phonopy = wf.create.phonopy.create_phonopy(
        structure=wf.relaxed_structure.outputs.structure,
        parameters=InputPhonopyGenerateSupercells(distance=displacement.run())
    )
    # print ('test: ', displacement.run())

    wf.check_consistency = wf.create.phonopy.check_consistency(phonopy=wf.phonopy.outputs.phonopy)
    wf.total_dos = wf.create.phonopy.get_total_dos(phonopy=wf.phonopy.outputs.phonopy)

    # iterate over all nodes, extract the log_output and store it in hdf5
    # control the amount of output via log_level

    return (
        wf.check_consistency,
        wf.total_dos,
        wf.relaxed_structure.outputs.out["energy"],
        wf.relaxed_structure.outputs.out["energy_initial"],
        wf.phonopy.outputs.out["energies"]
    )

In [5]:
wf = run_phonopy(element='Al', cell_size=3, vacancy_index=0, displacement=0.1)
wf.run()

energy:  0.8013167095854019 max_force:  9.367506770274758e-17
alles ok


{'imaginary_modes': False,
 'total_dos':      frequency_points  total_dos
 0           -0.604384        0.0
 1           -0.557641        0.0
 2           -0.510899        0.0
 3           -0.464156        0.0
 4           -0.417413        0.0
 ..                ...        ...
 196          8.557177        0.0
 197          8.603920        0.0
 198          8.650662        0.0
 199          8.697405        0.0
 200          8.744148        0.0
 
 [201 rows x 2 columns],
 'energy_relaxed': 0.8013167095854019,
 'energy_initial': 0.7996059979158101,
 'energy_displaced': 0     0.801477
 1     0.801456
 2     0.801444
 3     0.801514
 4     0.801395
 5     0.801562
 6     0.801523
 7     0.801430
 8     0.801585
 9     0.801368
 10    0.801554
 11    0.801399
 12    0.801477
 13    0.801477
 14    0.801477
 15    0.801441
 16    0.801515
 17    0.801450
 18    0.801505
 19    0.801477
 20    0.801478
 Name: energy, dtype: float64}

In [6]:
wf.phonopy.outputs.out.value['df']

,atoms,_internal,forces,energy,iter_index
0,"(Atom('Al', [0.007071067811865475, 2.018469473...",{'iter_index': [0]},"[[-0.023172401218310667, -0.02070281808992303,...",0.801477,[0]
1,"(Atom('Al', [-0.007071067811865475, 2.00432733...",{'iter_index': [1]},"[[0.023223919284408732, 0.017716773608982144, ...",0.801456,[1]
2,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [2]},"[[-0.00014853410846910053, -0.0045937310507149...",0.801444,[2]
3,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [3]},"[[0.00017067251834148624, 0.001684919320573907...",0.801514,[3]
4,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [4]},"[[8.120943301205726e-05, -0.001404462986394065...",0.801395,[4]
5,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [5]},"[[-8.291373798362776e-05, -0.00156522675623084...",0.801562,[5]
6,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [6]},"[[0.004142511131191316, -0.0014405213332437175...",0.801523,[6]
7,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [7]},"[[-0.004234841910250864, -0.001528501286112739...",0.801430,[7]
8,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [8]},"[[0.004360899946165295, -0.0014522923481715341...",0.801585,[8]
9,"(Atom('Al', [0.0, 2.011398405686311, 2.0113984...",{'iter_index': [9]},"[[-0.004498869592229898, -0.001516581248707151...",0.801368,[9]


In [7]:
xx

NameError: name 'xx' is not defined

### Pseudocode for output class

In [ ]:
@dataclass
class wfOutput:
    pass

    def keys(self):
        return self.__dict__.keys()

    def __getitem__(self, key):
        return self.__dict__[key]    

    def __call__(self, ...):
        ...

In [ ]:
class OutputEnergyStatic(wfOutput):
    import ase
    import pint
    
    energy = VarFunc(ase.get_energy, log_level=0, unit=pint.eV, ...)
    forces = VarFunc(ase.get_forces, log_level=1, unit=pint.eV/pint.Angstrom, ...)
    prop = VarType(ase.my_property, log_level=2, ...)

In [ ]:
def calc_static(atoms, output=OutputEnergyStatic()):
    atoms.calc = EMT()


    output['energy'] = atoms.get_energy()
    
    return output(atoms)

In [ ]:
def calc_static(atoms, output=wfOutput()):
    atoms.calc = EMT()

    output = dict()
    output['energy'] = atoms.get_energy()
    
    return output

In [ ]:
def calc_static(atoms):
    atoms.calc = EMT()

    output = wfOutput()
    output['energy'] = atoms.get_energy()
    
    return output

In [ ]:
wf.output = wf.create.calc_static.output(keys='energy')
wf.calc = wf.create.calc_static(output=wf.output)

In [ ]:
xx

In [ ]:
%%time
df = wf.iter(cell_size=list(range(1,4)), element=['Al'], vacancy_index=[None, 0], displacement=[0.01, 0.1]) #, Cu, Pd, Ag, Pt and Au])

In [ ]:
df

In [ ]:
df.energy_displaced

In [ ]:
xx

In [ ]:
@single_value_node()
def calc_static(atoms=None, engine=None): 
    print ('atoms: ', atoms)
    if engine is None:
        from ase.calculators.emt import EMT
        calculator = EMT() 

    atoms.calc = calculator

    out = {}
    # out['structure'] = atoms # not needed since identical to input
    out['forces'] = atoms.get_forces()
    out['energy'] = atoms.get_potential_energy()
  
    return out    

In [ ]:
#@single_value_node()
def generate_supercells(phonopy):
    from structuretoolkit.common import phonopy_to_atoms
    
    phonopy.generate_displacements()  

    supercells = [phonopy_to_atoms(s) for s in phonopy.supercells_with_displacements]
    return supercells

In [ ]:
@single_value_node()
def iter(node, qwargs):
    out = node.iter(**qwargs)
    return out

wf = Workflow('phonopy')
wf.structure = wf.create.pyiron_atomistics.Bulk('Al', cubic=True)
wf.supercell = wf.create.lammps.Repeat(structure = wf.structure, repeat_scalar=2)
wf.supercells = generate_supercells(structure=wf.supercell)
wf.calc = calc_static() 

*TODO: allowing to use nodes as input arguments in workflows is crucial but not (yet) available*
wf.df = iter(node=wf.calc, qwargs={'atoms': wf.supercells}) 

In [ ]:
@single_value_node()
def create_vacancy(structure, index: int|None = 0):
    structure = structure.copy()
    if index is not None:
        del structure[index]
    
    return structure   

In [ ]:
@Workflow.wrap_as.macro_node("structure")
def CubicBulkCell(wf, element: str, cell_size: int = 1, vacancy_index: int|None = None):
    wf.structure = wf.create.pyiron_atomistics.Bulk(
        name=element, cubic=True
    )
    wf.cell = wf.create.lammps.Repeat(structure = wf.structure, repeat_scalar=cell_size)

    wf.cell_with_vacancies = create_vacancy(structure=wf.cell, index=vacancy_index)
    return wf.cell_with_vacancies #.outputs.structure

In [ ]:
# The following function should be defined as a workflow macro (presently not possible)

@single_value_node()
def create_phonopy(structure, calculator=None, executor=None):
    from phonopy import Phonopy
    from structuretoolkit.common import atoms_to_phonopy
    
    phonopy = Phonopy(unitcell=atoms_to_phonopy(structure))  
    
    cells = generate_supercells(phonopy)# .run()
    gs = calc_static() 
    df = gs.iter(atoms=cells, executor=executor)
    phonopy.forces = df.forces
    
    return phonopy

In [ ]:
import numpy as np

structure = CubicBulkCell(element='Al', cell_size=2, vacancy_index=0).run().structure
phonopy = create_phonopy(structure).run()


In [ ]:
# @single_value_node()
# def create_phonopy(structure, calculator=None):
#     from phonopy import Phonopy
#     # from phonopy.structure.atoms import PhonopyAtoms
#     from structuretoolkit.common import phonopy_to_atoms, atoms_to_phonopy

#     if calculator is None:
#         from ase.calculators.emt import EMT
#         calculator = EMT()
    
#     phonopy = Phonopy(unitcell=atoms_to_phonopy(structure))  
#     phonopy.generate_displacements()
#     structures, forces, energies = [], [], []

#     for s in phonopy.supercells_with_displacements:
#         atoms = phonopy_to_atoms(s)
#         atoms.calc = calculator
#         structures.append(atoms)
#         forces.append(atoms.get_forces())
#         energies.append(atoms.get_potential_energy())
#     phonopy.forces = forces
    
#     return phonopy

In [ ]:
@single_value_node()
def get_dynamical_matrix(phonopy, q=[0,0,0]):
    import numpy as np
    
    if phonopy.dynamical_matrix is None:
        phonopy.produce_force_constants()
        phonopy.dynamical_matrix.run(q=q)
    dynamical_matrix = np.real_if_close(phonopy.dynamical_matrix.dynamical_matrix)
    # print (dynamical_matrix)
    return dynamical_matrix

In [ ]:
@single_value_node()
def get_eigenvalues(matrix):
    import numpy as np
    
    ew = np.linalg.eigvalsh(matrix)
    return ew

In [ ]:
@single_value_node()
def check_consistency(phonopy, tolerance=1e-10):
    dyn_matrix = get_dynamical_matrix(phonopy).run()
    ew = get_eigenvalues(dyn_matrix).run()
    # print ('ew: ', ew)
    ew_lt_zero = ew[ew < -tolerance]
    if len(ew_lt_zero) > 0:
        print (f'WARNING: {len(ew_lt_zero)} imaginary modes exist')
        has_imaginary_modes = True
    has_imaginary_modes = False 
    print ('alles ok')
    return has_imaginary_modes        

In [ ]:
@single_value_node()
def get_total_dos(phonopy, mesh=3*[10]):
    from pandas import DataFrame
    
    phonopy.produce_force_constants()
    phonopy.run_mesh(mesh=mesh)
    phonopy.run_total_dos()
    total_dos = DataFrame(phonopy.get_total_dos_dict())
    return total_dos

In [ ]:
wf = Workflow('phonopy')

wf.structure = CubicBulkCell(element='Al', cell_size=2, vacancy_index=None)
wf.phonopy = create_phonopy(structure=wf.structure.outputs.structure)  # explicit output needed since macro and not single_value_node (we should have also a sinle_value_macro)
#wf.dynamical_matrix = get_dynamical_matrix(phonopy=wf.phonopy)
wf.check_consistency = check_consistency(phonopy=wf.phonopy)
wf.total_dos = get_total_dos(phonopy=wf.phonopy)

wf.inputs_map = {'structure__cell_size': 'cell_size', 'structure__element': 'element'}
wf.outputs_map = {'total_dos__total_dos': 'total_dos', 
                  'check_consistency__has_imaginary_modes': 'imaginary_modes'
                 }

In [ ]:
wf.draw();

In [ ]:
%%time
df = wf.iter(cell_size=list(range(1,3)), element=['Al', 'Ni']) #, Cu, Pd, Ag, Pt and Au])

In [ ]:
df

In [ ]:
xx

In [ ]:
import matplotlib.pylab as plt

for n in range(2, 5):
    out = wf(supercell__repeat_scalar=n)
    df = out.total_dos #__total_dos
    n_atoms = len(wf.structure.outputs.structure.value)
    plt.plot(df.frequency_points, df.total_dos.values/n**3, label=(4*n**3, n_atoms))
plt.title('Phonon DOS')
plt.legend();

In [ ]:
%%time
import numpy as np
from pympipool import Executor



def calc(i, j, k):
    from mpi4py import MPI
    import time
    
    time.sleep(1)
    size = MPI.COMM_WORLD.Get_size()
    rank = MPI.COMM_WORLD.Get_rank()
    return np.array([i+rank, j, k]), size, rank

cores = 2
with Executor(cores_per_worker=cores, max_workers=1) as p:
    for i in range(0, 8, cores):
        fs = p.submit(calc, i, j=55, k=10)
        print(fs.result())

In [ ]:
%%time
from pympipool import Executor

def iter(func, max_workers=1, cores_per_worker=1, **kwargs):
    key = list(kwargs.keys())[0]
    val = kwargs[key]
    
    with Executor(cores_per_worker=cores_per_worker, max_workers=max_workers) as p:
        iter_dict = {'kwargs': kwargs}
        for i, n in enumerate(val):
            fs = p.submit(wf, **{key: n})
            iter_dict[i] = fs.result()
    return iter_dict        

out = iter(wf, supercell__repeat_scalar=list(range(1,3)))

In [ ]:
%%time
from pympipool import Executor

def iter(func, max_workers=1, cores_per_worker=1, **kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    with Executor(cores_per_worker=cores_per_worker, max_workers=max_workers) as p:
        #iter_dict = {'kwargs': kwargs}
        iter_dict = {}
        
        # Create an empty DataFrame to store the results
        df_result = pd.DataFrame(columns=keys)
    
        # Perform multidimensional for loop
        count = 0
        while indices[0] < lengths[0]:
            # Access the current elements using indices
            current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]
    
            # Add current_elements as a dictionary
            current_elements_kwarg = dict(zip(keys, current_elements))
            fs = p.submit(wf, **current_elements_kwarg)
            out = fs.result()
            iter_dict[count] = out
            count += 1

            for k, v in out.items():
                current_elements_kwarg[k] = v
                
            # Append the current_elements_kwarg to the DataFrame
            df_result = pd.concat([df_result, pd.DataFrame([current_elements_kwarg])], ignore_index=True)
    
            # Process the current elements (you can modify this part according to your needs)
            #print(f"Current Elements: {current_elements_kwarg}")
    
            # Update indices for the next iteration
            indices[num_dimensions - 1] += 1
    
            # Update indices and carry-over if needed
            for i in range(num_dimensions - 1, 0, -1):
                if indices[i] == lengths[i]:
                    indices[i] = 0
                    indices[i - 1] += 1
    return df_result        
       
df = iter(wf, cell_size=list(range(1,3)))

In [ ]:
%%time
df.to_hdf('test.h5', 'table')

In [ ]:
df.total_dos.values[1].plot();

In [ ]:
out[1]['total_dos'].plot(x='frequency_points');

In [ ]:
def multidimensional_for_loop(**kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    # Perform multidimensional for loop
    while indices[0] < lengths[0]:
        # Access the current elements using indices
        current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]

        # Process the current elements (you can modify this part according to your needs)
        print(f"Current Elements: {current_elements}")

        # Update indices for the next iteration
        indices[num_dimensions - 1] += 1

        # Update indices and carry-over if needed
        for i in range(num_dimensions - 1, 0, -1):
            if indices[i] == lengths[i]:
                indices[i] = 0
                indices[i - 1] += 1

    # You can add additional processing or return values as needed

# Example usage
list1 = [1, 2]
list2 = ['a', 'b', 'c']
list3 = [True, False]

multidimensional_for_loop(arg1=list1, arg2=list2, arg3=list3)

In [ ]:
def multidimensional_for_loop(**kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    # Perform multidimensional for loop
    while indices[0] < lengths[0]:
        # Access the current elements using indices
        current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]

        # Add current_elements as a keyword argument
        current_elements_kwarg = dict(zip(keys, current_elements))
        kwargs.update({'current_elements': current_elements_kwarg})

        # Process the current elements (you can modify this part according to your needs)
        print(f"Current Elements: {current_elements_kwarg}")

        # Update indices for the next iteration
        indices[num_dimensions - 1] += 1

        # Update indices and carry-over if needed
        for i in range(num_dimensions - 1, 0, -1):
            if indices[i] == lengths[i]:
                indices[i] = 0
                indices[i - 1] += 1

    # You can add additional processing or return values as needed

# Example usage
list1 = [1, 2]
list2 = ['a', 'b', 'c']
list3 = [True, False]

In [ ]:
multidimensional_for_loop(a=list1, b=list2, c=list3)

In [ ]:
import pandas as pd

def multidimensional_for_loop(**kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    # Create an empty DataFrame to store the results
    df_result = pd.DataFrame(columns=keys)

    # Perform multidimensional for loop
    while indices[0] < lengths[0]:
        # Access the current elements using indices
        current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]

        # Add current_elements as a dictionary
        current_elements_kwarg = dict(zip(keys, current_elements))

        # Append the current_elements_kwarg to the DataFrame
        df_result = pd.concat([df_result, pd.DataFrame([current_elements_kwarg])], ignore_index=True)

        # Process the current elements (you can modify this part according to your needs)
        print(f"Current Elements: {current_elements_kwarg}")

        # Update indices for the next iteration
        indices[num_dimensions - 1] += 1

        # Update indices and carry-over if needed
        for i in range(num_dimensions - 1, 0, -1):
            if indices[i] == lengths[i]:
                indices[i] = 0
                indices[i - 1] += 1

    # You now have a DataFrame containing the results
    print("DataFrame Result:")
    print(df_result)

    # You can add additional processing or return values as needed

# Example usage
list1 = [1, 2]
list2 = ['a', 'b', 'c']
list3 = [True, False]

multidimensional_for_loop(a=list1, b=list2, arg3=list3)

In [ ]:
from dataclasses import dataclass
from typing import Optional, Union

@dataclass
class InputPhonopyGenerateSupercells:
    distance: float = 0.01
    is_plusminus: Union[str, bool] = "auto"
    is_diagonal: bool = True
    is_trigonal: bool = False
    number_of_snapshots: Optional[int] = None
    random_seed: Optional[int] = None
    temperature: Optional[float] = None
    cutoff_frequency: Optional[float] = None
    max_distance: Optional[float] = None

    def keys(self):
        return self.__dict__.keys()
        
    def __getitem__(self, key):
        return self.__dict__[key]

In [ ]:
par = InputPhonopyGenerateSupercells(distance=0.02)

In [ ]:
def f(distance=0.01, **kwargs):
    print (kwargs)
    print (distance)

In [ ]:
f(**par)

In [ ]:
f(**{'distance': 0.03})

In [ ]:
dict_lst = {}
for count in range(1, 4):
    current_elements_kwarg = {'a': 1}
    _internal = {}
    _internal["iter_index"] = count
    current_elements_kwarg["_internal"] = _internal
    print("indices: ", _internal)    

    for k, v in current_elements_kwarg.items():
        if count == 1:
             dict_lst[k] = [v]
        else:    
            if k in dict_lst:
                dict_lst[k].append(v)
            else:
                ValueError(f"New key appears at count {count}")      
    
dict_lst